# Create a Meeting Report from an Audio File

This project is inspired by a Udemy course titled ["AI Engineer Core Track: LLM Engineering, RAG, QLoRA, Agents"](https://www.udemy.com/course/llm-engineering-master-ai-and-large-language-models/), where the instructor used minutes from a Denver City Council meeting. However, I adapted the project to make it applicable to meetings in French, such as those of the Board of Directors of the Drummond Junior Chamber of Commerce.

I used an audio recording of a meeting of the Drummond Junior Chamber of Commerce for testing, but since this meeting is confidential, you can download excerpts from the minutes of a Denver meeting and select a portion of the meeting to transcribe. You can download it here:  
https://drive.google.com/file/d/1N_kpSojRR5RYzupz6nqM8hMSoEF_R7pU/view?usp=sharing

If you prefer to work with the original data, the HuggingFace dataset is available [here](https://huggingface.co/datasets/huuuyeah/meetingbank), and the audio can be downloaded [here](https://huggingface.co/datasets/huuuyeah/MeetingBank_Audio/tree/main).

The goal of this project is to use the audio to generate a meeting report, including action items.

For this project, you can either use the minutes from the Denver meeting or record something yourself!


## Run on GPU:

**Tip:**

Ideally, for better performance, it is recommended to run the project on a GPU. Make sure to install PyTorch with GPU support. You can check if a GPU is available with the following command:


In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)
  print("Connected to a GPU")

Tue Dec 23 07:37:49 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 577.03                 Driver Version: 577.03         CUDA Version: 12.9     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 3060 ...  WDDM  |   00000000:01:00.0  On |                  N/A |
| N/A   52C    P8             12W /   95W |     476MiB /   6144MiB |      6%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
import torch

# Check if we are running on GPU
device = "cuda" if torch.cuda.is_available() else "cpu"

print(f"Using device: {device}")

Using device: cuda


In [ ]:
# imports

import os
import requests
import subprocess
from IPython.display import Markdown, display, update_display
from openai import OpenAI
from huggingface_hub import login
from transformers import AutoTokenizer, AutoModelForCausalLM, TextStreamer, BitsAndBytesConfig
from transformers import pipeline
from datetime import datetime
# import torch # already imported above

In [4]:
# Constants

LLAMA = "meta-llama/Llama-3.2-3B-Instruct"

In [5]:
# Sign in to HuggingFace Hub

hf_token = os.getenv("HF_TOKEN")
login(hf_token, add_to_git_credential=True)

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


# STEP 1 : Convert video to audio

In [6]:
video_filename = "2025-12-15 11-58-31 - Copie.mkv"
video_fullpath = "videos/" + video_filename

In [7]:
# Dynamically set audio_filename based on video filename
audio_filename = os.path.basename(video_filename).replace(".mkv", ".mp3")
audio_fullpath = "audio/" + audio_filename

print(f"Audio filename: {audio_filename}")
print(f"Audio will be saved to: {audio_fullpath}")

Audio filename: 2025-12-15 11-58-31 - Copie.mp3
Audio will be saved to: audio/2025-12-15 11-58-31 - Copie.mp3


In [61]:
result = subprocess.run(
    ["ffmpeg", "-i", video_fullpath, "-q:a", "0", "-map", "a", audio_fullpath, "-y"],
    capture_output=True,
    text=True
)

# STEP 2 : Split audio in smaller audios

In [9]:
# Check length of audio file (in minutes)
result = subprocess.run(
    ["ffprobe", "-v", "error", "-show_entries", "format=duration", "-of", "default=noprint_wrappers=1:nokey=1", audio_fullpath],
    capture_output=True,
    text=True
)

if result.returncode == 0:
    duration_seconds = float(result.stdout.strip())
    duration_minutes = duration_seconds / 60
    print(f"Durée de l'audio : {duration_minutes:.2f} minutes")
else:
    print("Erreur lors de la récupération de la durée de l'audio")
    print(result.stderr)

Durée de l'audio : 56.84 minutes


In [26]:
SPLIT_THRESHOLD = 10 # minutes

# Chunk audio file if longer than 15 minutes (in 15 minutes segments)
audio_filenames = []

if duration_minutes > SPLIT_THRESHOLD:
    num_chunks = int(duration_minutes // SPLIT_THRESHOLD) + (1 if duration_minutes % SPLIT_THRESHOLD > 0 else 0)
    for i in range(num_chunks):
        start_time = i * SPLIT_THRESHOLD * 60
        output_chunk = audio_fullpath.replace(".mp3", f"_part{i+1}.mp3")
        audio_filenames.append(output_chunk)
        subprocess.run(
            ["ffmpeg", "-i", audio_fullpath, "-ss", str(start_time), "-t", str(SPLIT_THRESHOLD * 60), output_chunk, "-y"],
            capture_output=True,
            text=True
        )
        print(f"Created chunk: {output_chunk}")
else:
    audio_filenames.append(audio_fullpath)

Created chunk: audio/2025-12-15 11-58-31 - Copie_part1.mp3
Created chunk: audio/2025-12-15 11-58-31 - Copie_part2.mp3
Created chunk: audio/2025-12-15 11-58-31 - Copie_part3.mp3
Created chunk: audio/2025-12-15 11-58-31 - Copie_part4.mp3
Created chunk: audio/2025-12-15 11-58-31 - Copie_part5.mp3
Created chunk: audio/2025-12-15 11-58-31 - Copie_part6.mp3


# STEP 3 : Transcript the audio

## Option 1 : Use Open Source for Transcription - Hugging Face Pipelines

In [66]:
# Define pipeline
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Pipeline Whisper utilisera: {device}")

pipe = pipeline(
    "automatic-speech-recognition",
    model="openai/whisper-medium",
    torch_dtype=torch.float16 if device == "cuda" else torch.float32,
    device=device,
    return_timestamps=True
)

Pipeline Whisper utilisera: cuda


Device set to use cuda


In [77]:
transcripts = []
transcript_filenames = []
for audio_filename in audio_filenames:
    print(f"Processing file: {audio_filename}")
    result = pipe(audio_filename, generate_kwargs={"language": "french"})
    transcription = result["text"]
    print(transcription)

    # Save transcription to a text file (in transcript folder)
    transcript_folder = "transcripts"
    os.makedirs(transcript_folder, exist_ok=True)
    transcript_filename = os.path.join(transcript_folder, os.path.basename(audio_filename).replace(".mp3", ".txt"))

    with open(transcript_filename, "w", encoding="utf-8") as f:
        f.write(transcription)
    transcripts.append(transcription)
    transcript_filenames.append(transcript_filename)

Processing file: audio/2025-12-15 11-58-31 - Copie_part1.mp3


KeyboardInterrupt: 

In [ ]:
display(Markdown(transcription))

## Option 2: Use OpenAI for Transcription

In [27]:
print(audio_fullpath)

audio/2025-12-15 11-58-31 - Copie.mp3


In [12]:
audio_file = open(audio_fullpath, "rb")

In [28]:
# Open files
audio_files = []
for filename in audio_filenames:
    audio_file = open(filename, "rb")
    audio_files.append(audio_file)

In [34]:
# Sign in to OpenAI

AUDIO_MODEL = "gpt-4o-mini-transcribe"

openai = OpenAI()

transcripts = []
transcript_filenames = []

for audio_file, audio_filename in zip(audio_files, audio_filenames):
    transcription = openai.audio.transcriptions.create(
        model=AUDIO_MODEL, 
        file=audio_file, 
        response_format="text", 
        language="fr",
    )
    
    # Save transcription to a text file (in transcript folder)
    transcript_folder = "transcripts"
    os.makedirs(transcript_folder, exist_ok=True)
    transcript_filename = os.path.join(transcript_folder, os.path.basename(audio_filename).replace(".mp3", ".txt"))

    with open(transcript_filename, "w", encoding="utf-8") as f:
        f.write(transcription)
    transcripts.append(transcription)
    transcript_filenames.append(transcript_filename)


In [30]:
print(transcription)

Bon, là, on va le dire, je vais encore le dire. Mais je suis dans la bulle de neige, pas de problème. Bon, on va trop regarder. Salut. Bonjour. Salut, ça, salut Tom. Salut. Allô tout le monde. Parfait, vous êtes arrivé au moment où Fred disait « je veux vivre, je m'en calisse ». Ça a fait avec nos nouvelles valeurs. Salut Kev. Salut. Tu vas nous manquer, Maud et Pamela. Un gros comité. Oui, c'est le cas. Non, mais oui et non. En fait, Kev est plus là en termes d'intérêt de passation des savoirs et tout. Parce que c'était le trésorier l'an passé. Mais est-ce que tu vas prendre réellement la gestion de certains dossiers? Je ne sais pas. Ça sera à voir. Isabelle est là à titre d'information parce que c'est elle qui détient l'information. Mais elle n'aura pas à prendre position nécessairement à gérer les dossiers non plus. Ça va être plus dans notre cas à nous. Ça va faire un beau background? Un peu. Oui. C'est mon salon dans le fond. On a tous la même chose. On a tous la même chose. Je su

In [35]:
display(Markdown(transcription))

Dans celui-là, je mettrais un de vous deux dans celui financier, du moins un qui connaît même les deux modes. Je ne sais pas si tu veux être... Maud, je pense qu'il est mieux d'être dans l'ego. Parce que justement... Moi, je me verrais moins financier et Lego, sinon ceux que je me verrais le plus, je pense. Parfait. Moi, je prendrais peut-être gouvernance et stratégie, mettons. Et qui colle plus à mon... Je ne sais pas. Puis il reste quand même qu'on va tous les lire, là. On va apporter notre grain de sel, même si on n'a pas ces deux... On peut-tu dans Drive? Oui, on pourrait faire un... Je mettrais un template, là. OK. Est-ce qu'on met, mettons, tout le temps Maude avec genre Tom, vous faites les deux même, comme ça c'est plus simple en termes de disponibilité? Qu'est-ce que tu en penses? Moi, je pense que... Légal, peut-être je le laisserais à Fred, non? Je ne sais pas. Oui, OK, oui. Mais là, à la limite, je peux enlever mon nom si j'étais avec Tommy, ça ne me dérange pas, là. Non, non, mais je disais ça de même, là. C'est parce que vous étiez les deux premiers dans ma face, dans mon écran. C'est pas obligé d'être... En fait, ça ne me dérange pas, c'est plus vous en termes de disponibilité, ce que vous préférez, tu sais, gérer une rencontre avec une autre personne, puis travailler les deux, ou gérer deux petites rencontres. Bien, est-ce que... Je ne sais pas vraiment que vous voyez ça. On allait faire, mettons, une rencontre, ou c'est plus qu'on travaille les deux chacun de notre côté, après ça, on fait une petite rencontre juste pour se donner nos points, où on se rencontre pour l'écrire ensemble. Ça va dépendre de chaque personne, mais... Oui. On laisse les duos gérer ça. OK. En tout cas, moi, je peux faire la réputation, là. Oui, je pense que c'est pas mal. Je pense que cette ligne-là... Ça fait du sens, effectivement. Oui, mais je pense que je peux faire aussi le... Je peux être dans les mêmes relations stratégiques, si ça ne me dérange pas. C'est bon. Là, j'écris tous les noms, là. Oui, oui, c'est bon. Je suis en train de... Kev, Tommy, vous, ça serait plus quoi? Moi, je suis dans n'importe quoi. Moi, je réalise que c'est où qu'il n'y a pas d'intérêt, je vais y aller. Bien, moi, c'est l'ego, là. On dirait que je connais un peu moins ça. Est-ce que... Je trouve, mettons, Tom, de parler des questions que tu as posées puis de challenger en termes de gouvernance, j'aimerais ça peut-être travailler avec toi. Challenge. Pas tous en même temps, surtout. Je vais parler financier avec Maud. Parfait. L'ego, c'est bon. On va parler de duo. Oui. Je vais suivre ton... Fait que là, je préfère quoi, là? Ce qui a opérationnel. OK. Il manque de monde. Sinon, moi, je peux aller dans la réputationnelle. Avec Pam. On a deux personnes partout. Il manque humain. Il manque quelqu'un avec moi. Je vais te rejoindre. D'accord. OK. Fait qu'on aurait deux personnes partout. Ça fait qu'on travaille avec différentes personnes. On n'est pas tout le temps en duo. Je pense que ça ne sera pas mauvais non plus. Ce que je vais faire, dans le fond, de mon côté, puis après ça, je vous laisse aller, je vais mettre cette liste-là dans le drive. Je vous ferai le lien. En tout cas, je vous dirai où dans le Google Drive. Je vais vous mettre un exemple de tableau. Puis après ça, s'il y a des ajouts, vous pensez à d'autres risques, vous en regroupez, il n'y a pas de stress. On gère notre section comme on veut. Je nous ferai peut-être une rencontre, mettons, mi-janvier de suivi pour pas au prochain CA, mais l'autre, qu'on ait une matrice relativement complète. Je ne sais pas si ça ferait du sens pour vous. Mi-janvier. Mi-janvier. Bien, c'est laissé les fêtes. On se regrouperait, nous, pour regarder ça, rendre ça plus... être sûr qu'on est confortable avec tout ce qui est là, qu'on n'oublie rien. Après ça, il y a une présentation au CA, pas celui de janvier, mais l'autre. OK. Parfait. Ça ferait du sens. Ça fait du sens pour moi. Parfait. Parce que, bon, en même temps, je sais qu'avec les vacances de Noël, on n'a pas le goût nécessairement de travailler là-dessus. On ne se cachera pas. Puis c'est que la dernière semaine de janvier, il y a beaucoup d'événements. Fait que je me dis qu'on pourrait se donner du lousse. Parfait. Excellent. Fait que moi, je m'occupe de mettre ça dans le drive d'ici le week-end avec un exemple de matrice. Puis je m'occupe aussi de revenir sur les questions du budget dans le Slack. Je nous envoie un doodle pour qu'on se rassoie virtuellement en janvier. On verra le meilleur moment. Puis après ça, on aura des présentations à faire au CA d'après. Parfait. Cool. Ça fut efficace. Oui. Merci pour votre participation. Julie, je viens de t'envoyer un code sur ton cell. Ah oui, j'ai reçu. Bien, en fait, si vous voulez quitter le parérieur, on va le gérer pour vous. Bye-bye. Bye. Bye.


In [89]:
# Count number of words returned
print("Number of tokens in transcription:")
print(len(transcription.split()))


Number of tokens in transcription:
1468


## Option 3: Use Groq for Transcription (Whisper)

In [83]:
# Use Groq / Whisper
groq_url = "https://api.groq.com/openai/v1"
groq_api_key = os.getenv('GROQ_API_KEY')
groq = OpenAI(api_key=groq_api_key, base_url=groq_url)

model = "whisper-large-v3"
audio_file = audio_files[0]
transcription = groq.audio.transcriptions.create(model=model, file=audio_file, response_format="text", language="fr")
print(transcription)

 ... Merci. Bon, honnêtement, moi j'ai un VUS, je vais en call this. Honnêtement, je vais rentrer dans la bulle de démarrage, bad. Pas de problème. Bon, pas trop regardant. Salut! Bonjour! Salut Isaac, salut Tom. Salut tout le monde. Parfait, vous êtes arrivé au moment où Fred disait j'ai un VUS, Je m'en calais, c'est excellent. Ça fait avec nos nouvelles valeurs. Salut Kev. Salut. Ce qui va nous manquer, Maude puis Pamela. Un gros comité. Oui, c'est le C. Non, bien, oui puis non. En fait, Kev est plus là en termes d'intérêt de passation des savoirs et tout, parce que c'était le trésorier l'an passé, mais est-ce que tu vas prendre réellement la gestion de certains dossiers, je ne sais pas, ça sera à voir. Isabelle est là à titre d'information parce que c'est elle qui détient l'information, mais elle n'aura pas à prendre position nécessairement à gérer les dossiers non plus, ça va être plus dans notre cas à nous. Vous avez un beau background. Un peu. C'est mon salon, dans le fond. On a 

# STEP 4: Load agenda as template for the minutes (optionnal)

In [59]:
agenda_path = "agendas/Ordre du jour du 2 décembre 2025.docx"

In [62]:
document = Document(agenda_path)

In [91]:
print(document.paragraphs[29].text)
# print("\n")
print(document.paragraphs[30].text)
print(document.paragraphs[30].style)
print(document.paragraphs[30].style.name)
print(document.paragraphs[30]._p.pPr.numPr.ilvl.val)

Suivis financiers
Présentation et adoption des chiffres au 31 octobre 2025 (Maude et Isabelle au besoin)
_ParagraphStyle('Normal') id: 2554444654160
Normal
1


In [10]:
def get_list_level(para):
    pPr = para._p.pPr
    if pPr is not None and pPr.numPr is not None:
        ilvl = pPr.numPr.ilvl
        if ilvl is not None:
            return int(ilvl.val)
    return None


In [9]:
from docx.oxml.ns import qn

def get_paragraph_text_with_links(para):
    """
    Retourne le texte visible du paragraphe,
    incluant le texte des hyperliens (sans l'URL).
    """
    texts = []

    for child in para._p:
        # Texte normal (runs)
        if child.tag == qn("w:r"):
            for node in child:
                if node.tag == qn("w:t") and node.text:
                    texts.append(node.text)

        # Texte dans un hyperlien
        elif child.tag == qn("w:hyperlink"):
            for sub in child:
                if sub.tag == qn("w:r"):
                    for node in sub:
                        if node.tag == qn("w:t") and node.text:
                            texts.append(node.text)

    return "".join(texts).strip()


In [8]:
from docx import Document

def get_list_level(para):
    pPr = para._p.pPr
    if pPr is not None and pPr.numPr is not None:
        ilvl = pPr.numPr.ilvl
        if ilvl is not None:
            return int(ilvl.val)
    return None


def docx_to_markdown(docx_path):
    document = Document(docx_path)
    markdown_lines = []

    counters = [0, 0, 0, 0, 0]

    for para in document.paragraphs:
        raw_text = para.text.strip()
        if not raw_text:
            continue

        # Titres
        if para.style.name.startswith("Heading"):
            level = int(para.style.name.replace("Heading ", ""))
            markdown_lines.append("#" * level + " " + raw_text)
            continue

        # Listes numérotées Word (agenda)
        list_level = get_list_level(para)
        if list_level is not None:
            counters[list_level] += 1
            for i in range(list_level + 1, len(counters)):
                counters[i] = 0

            number = ".".join(str(counters[i]) for i in range(list_level + 1))

            # Reconstruction avec gras (si possible)
            formatted_text = ""
            for run in para.runs:
                if run.bold:
                    formatted_text += f"**{run.text}**"
                else:
                    formatted_text += run.text

            # 🔑 FALLBACK hyperliens / cas vides
            if not formatted_text.strip():
                formatted_text = raw_text

            markdown_lines.append(f"{number}. {formatted_text.strip()}")
            continue

        # Listes à puces
        if "List Bullet" in para.style.name:
            formatted_text = ""
            for run in para.runs:
                if run.bold:
                    formatted_text += f"**{run.text}**"
                else:
                    formatted_text += run.text

            if not formatted_text.strip():
                formatted_text = raw_text

            markdown_lines.append(f"- {formatted_text.strip()}")
            continue

        # Texte normal
        formatted_text = ""
        for run in para.runs:
            if run.bold:
                formatted_text += f"**{run.text}**"
            else:
                formatted_text += run.text

        if not formatted_text.strip():
            formatted_text = raw_text

        markdown_lines.append(formatted_text.strip())

    return "\n\n".join(markdown_lines)

In [99]:
agenda_markdown = docx_to_markdown(agenda_path)

display(Markdown(agenda_markdown))

**Jeune Chambre de Drummond**

**Réunion régulière du conseil d’administration**

**Date : Mardi 2 décembre 2025**

**Heure : 7h00 à 9h00**

**Lieu : Service Signature Desjardins, 905 rue Gauthier, Drummondville. **

**Rappel : **

**Mission **: Favoriser le développement des compétences et du réseau d’affaires des entrepreneurs et professionnels âgés entre 18 et 40 ans de la MRC de Drummond.

**Vision **: Nous visons à être la référence des jeunes gens d’affaires en étant reconnu comme un acteur d’influence auprès du milieu des affaires, en développant des partenariats stratégiques et en contribuant         au rayonnement de ses membres par la force de son réseau, son dynamisme et son audace.

**Valeurs **: accessibilité, leadership, intégrité, collaboration & engagement

**« Proposition d’ordre du jour »**

1. Ouverture de la réunion régulière et constatation du quorum (Julie)

2. Lecture et adoption de l’ordre du jour (Julie)

3. Lecture et adoption du (Julie)

3.1. Suites au procès-verbal (Julie)

3.1.1. Code d’éthique et déclaration de conflit d’intérêts

3.1.2. Suivis des jumelages

3.1.3. Tour de table objectifs 2025-26

3.1.4. Comités de C.A. Et 1er mandat de chacun

3.1.5. Rencontre individuelle décembre

4. Suivis financiers

4.1. Présentation et adoption des chiffres au 31 octobre 2025 (Maude et Isabelle au besoin)

4.1.1. Bilan au 31 octobre 2025

4.1.2. État des résultats au 31 octobre 2025

4.2. Présentation et adoption des  (Maude et Isabelle au besoin)

5. Rapport de la DG

5.1. Membership (Isabelle)

5.2. Suivis des comités bénévoles depuis le 14 octobre 2025 (Isabelle)

5.3. Rétroaction des événements de septembre, octobre et novembre 2025 :(Isabelle)

5.3.1. Revenus et dépenses par événements

5.4. Événements à venir (Isabelle)

5.5. Réunions et représentations externes (Isabelle)

6. Implication de la JCD (Julie)

6.1. Organismes et causes sociales

6.2. Sonder nos membres

6.3. Prendre position enjeu régional

7. Varia

7.1. Moitié-moitié (Julie)

8. Huis clos (C.A + CCID)

9. Huis clos (C.A. Seulement)

10. Levée de la réunion

# STEP 5: Analyze & Report

In [11]:
# Load transcripts from transcript files
transcript_base_filename = "2025-12-15 11-58-31 - Copie"
transcript_folder = "transcripts"

transcripts = []
for filename in os.listdir(transcript_folder):
    if filename.startswith(transcript_base_filename) and filename.endswith(".txt"):
        filepath = os.path.join(transcript_folder, filename)
        with open(filepath, "r", encoding="utf-8") as f:
            transcript = f.read()
            transcripts.append(transcript)

In [12]:
len(transcripts)

6

In [13]:
# Merge all transcripts into a single text
full_text = "".join(transcripts)

In [ ]:
display(Markdown(full_text))

In [14]:
# Tokenize the full text and count tokens
tokenizer = AutoTokenizer.from_pretrained(LLAMA)
tokens = tokenizer(full_text)
num_tokens = len(tokens['input_ids'])
print(f"Number of tokens in full transcription: {num_tokens}")

Number of tokens in full transcription: 11527


In [ ]:
current_date = datetime.now().strftime("%y-%m-%d")
system_message = f"""
Current date: {current_date}.
You produce minutes of meetings from transcripts, with summary, key discussion points,
takeaways, votes, and action items with board members and general manager, in markdown format without code blocks, in french.
"""

user_prompt = f"""
Below is an extract transcript of the C.A. of the Jeune Chambre de Drummond.
Please write minutes in markdown without code blocks, including:
- a summary with attendees, date, location
- discussion points
- takeaways
- votes
- action items with board members and general manager.
If a section is not applicable, say so.
If a vote is taken, please write it in the format:
**Résolution [current_date yy-mm-dd]-[incremental_number]**
Il est proposé par [name_1] et appuyé par [name_2] de [description of the motion].
**-Adopté à l'unanimité-**
Example:
**Résolution 25-12-31-01**
Il est proposé par Frédéric et appuyé par Laurianne d'ouvrir la réunion à 7h03. Le quorum est constaté.
**-Adopté à l'unanimité-**
======
"""

if 'agenda_markdown' in locals() and agenda_markdown.strip():
    user_prompt += f"""
Use the agenda to help structure the minutes. Here is the agenda:
{agenda_markdown}
"""

user_prompt += f"""
Here is the transcript:
{full_text}
"""

messages = [
    {"role": "system", "content": system_message},
    {"role": "user", "content": user_prompt},
]

## Option A : Using LLAMA and HuggingFace

In [14]:
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_quant_type="nf4",
)

In [15]:
tokenizer = AutoTokenizer.from_pretrained(LLAMA)
tokenizer.pad_token = tokenizer.eos_token
inputs = tokenizer.apply_chat_template(messages, return_tensors="pt").to("cuda")
streamer = TextStreamer(tokenizer)
model = AutoModelForCausalLM.from_pretrained(
    LLAMA,
    device_map="auto",
    quantization_config=quant_config
)
outputs = model.generate(inputs, max_new_tokens=2000, streamer=streamer)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 19 Dec 2025

You produce minutes of meetings from transcripts, with summary, key discussion points,
takeaways, votes, and action items with board members and general manager, in markdown format without code blocks, in french.<|eot_id|><|start_header_id|>user<|end_header_id|>

Below is an extract transcript of the C.A. of the Jeune Chambre de Drummond.
Please write minutes in markdown without code blocks, including:
- a summary with attendees, date, location
- discussion points
- takeaways
- votes
- action items with board members and general manager.

Here is the transcript:
Bon, là, moi, j'avais bien un chocolatiste, là. Mais je suis dans la bulle de neige. Pas de problème. Bon, pas trop regardé. Salut. Bonjour. Salut, ça, salut, Tom. Salut. Allô, tout le monde. Parfait, vous êtes arrivés au moment où Fred disait « j'avais un VUS, je m'en câlisse ». Ça a fait avec nos nouvell

OutOfMemoryError: CUDA out of memory. Tried to allocate 12.16 GiB. GPU 0 has a total capacity of 6.00 GiB of which 1.27 GiB is free. Of the allocated memory 3.61 GiB is allocated by PyTorch, and 92.19 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [30]:
# Decode the output (Without the prompt)
generated_text = tokenizer.decode(outputs[0][len(inputs[0]):], skip_special_tokens=True)
generated_text = generated_text.replace("assistant", "", 1).strip()  # Enlève seulement le premier "assistant"

## Option B : Using OpenAi GPT

In [16]:
model = "gpt-4o-mini"

In [17]:
openai = OpenAI()
response = openai.chat.completions.create(
    model=model,
    messages=messages, 
)

In [18]:
generated_text = response.choices[0].message.content

In [19]:
display(Markdown(generated_text))

## Minutes de la réunion

**Date :** [à déterminer]  
**Lieu :** [à déterminer]  
**Participants :** Fred, Tom, Kevin, Maud, Pamela, Isabelle, autres membres du CA

### Résumé
La réunion du conseil d'administration de la Jeune Chambre de Drummond a principalement porté sur la révision du budget et la gestion des risques. Des questions ont été soulevées concernant l'augmentation du membership par rapport à l'événementiel, ainsi que la nécessité de mettre en place une politique de gestion des risques.

### Points de discussion
- Examen du budget présenté le 3 décembre.
- Augmentation du membership vs augmentation des événements.
- Nécessité d'une approche conservatrice pour la facturation des événements.
- Stratégies pour accroître la rentabilité des événements.
- Gestion des risques et mise en place d'une matrice de risques.
- Importance de documenter les décisions et les votes par courriel.

### Points à retenir
- La situation financière nécessite des clarifications sur les revenus par rapport aux dépenses, en particulier en ce qui concerne les commandites.
- L'absence de financement additionnel pourrait impacter le budget global.
- La nécessité d'une matrice de gestion des risques pour une meilleure gouvernance et protection de l'organisation a été soulignée.

### Votes
**Résolution [date non spécifiée]-01**  
Il est proposé par Fred et appuyé par Maud d'approuver l'envoi de la résolution par courriel concernant le budget révisé.  
**-Adopté à l'unanimité-**

### Actions à entreprendre
- **Fred :** Partager les résultats des discussions budgétaires avec le CA sur Slack et organiser un vote par courriel.
- **Isabelle :** Préparer et fournir la matrice de gestion des risques pour une discussion future.
- **Tous les membres :** Se répartir en sous-groupes pour travailler sur les catégories de risques assignées.
- **Tom :** Mettre en place un tableau dans Google Drive pour la matrice des risques.
- **Réunion de suivi :** Prévoir une rencontre virtuelle en janvier pour faire le point sur les risques.

### Prochaine réunion
- Prévoir une réunion de suivi pour mi-janvier pour passer en revue les risques identifiés et préparer la présentation pour le prochain CA.

Merci à tous pour votre participation et votre engagement!

In [20]:
# Save the minutes to a markdown file (in minutes/markdown folder)
minutes_folder = "minutes/markdown"
os.makedirs(minutes_folder, exist_ok=True)
minutes_filename = os.path.join(minutes_folder, os.path.basename(audio_filename).replace(".mp3", ".md"))
with open(minutes_filename, "w", encoding="utf-8") as f:
    f.write(generated_text)

## Convertir en word

In [23]:
from docx import Document
from docx.shared import Pt, Inches
from docx.enum.text import WD_ALIGN_PARAGRAPH

In [24]:
# Créer un document Word
doc = Document()

# Titre
title = doc.add_heading('Compte-rendu de réunion', 0)
title.alignment = WD_ALIGN_PARAGRAPH.CENTER

# Ajouter le contenu généré
# Si le texte contient des sections markdown, on peut les parser
lines = generated_text.split('\n')

for line in lines:
    line = line.strip()
    if not line:
        continue
    
    # Détection des titres markdown
    if line.startswith('###'):
        doc.add_heading(line.replace('###', '').strip(), level=3)
    elif line.startswith('##'):
        doc.add_heading(line.replace('##', '').strip(), level=2)
    elif line.startswith('#'):
        doc.add_heading(line.replace('#', '').strip(), level=1)
    # Détection des textes entourés de **
    elif line.startswith('**') and line.endswith('**'):
        doc.add_heading(line.replace('**', '').strip(), level=3)
    # Détection des listes
    elif line.startswith('- ') or line.startswith('* '):
        doc.add_paragraph(line[2:], style='List Bullet')
    # Texte normal
    else:
        doc.add_paragraph(line)

# Sauvegarder le document
minutes_word_folder = "minutes/word"
os.makedirs(minutes_word_folder, exist_ok=True)
word_filename = os.path.join(minutes_word_folder, os.path.basename(audio_filename).replace(".mp3", ".docx"))
doc.save(word_filename)
print(f"Document Word sauvegardé: {word_filename}")

Document Word sauvegardé: minutes/word\2025-12-15 11-58-31 - Copie.docx
